## Import Libraries

In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

2024-11-20 20:21:01.025027: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-20 20:21:01.028068: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-20 20:21:01.036826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732134061.051143   10363 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732134061.055378   10363 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 20:21:01.071753: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## Acquire Training Data

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
text = text[0:len(text)//2] # only use half of the dataset for speed

## Data Processing

In [8]:
characters = sorted(set(text))
charToIndex = dict((c, i) for i, c in enumerate(characters))
indexToChar = dict((i, c) for i, c in enumerate(characters))

SEQ_LENGTH = 40 # setence length
STEP_SIZE = 3 # num of characters to step when creating a new sentence

sentences = []
next_char = []

for i in range(0, len(text)-SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i:i+SEQ_LENGTH])
    next_char.append(text[i+SEQ_LENGTH])


print(f'text length: {len(text)}, num of distinct characters: {len(characters)}')

text length: 557697, num of distinct characters: 37


In [4]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

for i in range(len(sentences)):
    for j in range(SEQ_LENGTH):
        x[i][j][charToIndex[sentences[i][j]]] = 1
    y[i][charToIndex[next_char[i]]] = 1

print(x)
print(y)

[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False  True False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False  True False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False  True Fal

## Model Building

In [5]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01))
model.fit(x, y, batch_size=256, epochs=4)

2024-11-20 20:21:06.791877: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/workspaces/NeuralNetworkTextGeneration/Movie_Reviews_Generator/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/4


2024-11-20 20:21:07.227921: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 275111280 exceeds 10% of free system memory.


727/727 ━━━━━━━━━━━━━━━━━━━━ 71s 97ms/step - loss: 2.4550
Epoch 2/4
727/727 ━━━━━━━━━━━━━━━━━━━━ 86s 103ms/step - loss: 1.7330
Epoch 3/4
727/727 ━━━━━━━━━━━━━━━━━━━━ 76s 95ms/step - loss: 1.5778
Epoch 4/4
727/727 ━━━━━━━━━━━━━━━━━━━━ 81s 94ms/step - loss: 1.5020


## Generate Text

In [12]:
def sample(preds, temparture=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temparture
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ""
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    print(len(sentence))
    
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            print(t)
            x_predictions[0, t, charToIndex[char]] = 1
            
        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = indexToChar[next_index]
        
        generated += next_character
        sentence = sentence[1:] + next_character
        
    return generated
        

In [13]:
print(generate_text(10, 0.2))
# print(generate_text(300, 0.2))
# print(generate_text(300, 0.4))
# print(generate_text(300, 0.5))
# print(generate_text(300, 0.6))
# print(generate_text(300, 0.7))
# print(generate_text(300, 0.8))

40
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3